# Predicting House Sale Prices

We will be working with housing data for the city of Ames, Iowa, United States from 2006 to 2010 in order to predict house sale prices. You can also read about the different columns in the data [here](https://s3.amazonaws.com/dq-content/307/data_description.txt).

In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

In [86]:
housing = pd.read_csv('AmesHousing.tsv', delimiter='\t')

housing.shape

(2930, 82)

In [87]:
def transform_features(df):
    return df

def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    
    # Hardcoded middle
    train = df[:1460]
    test = df[1460:]
    
    # Only capturing data good for linear models
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    # SalePrice is the target feature
    features = numeric_train.columns.drop("SalePrice")
    
    # Train model
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    
    # Tests model
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

train_and_test(transform_features(select_features(housing))) # Select then transform then evaluate

57088.25161263909

## Feature Engineering

Here we will do the following:

* Remove features that we don't want to use in the model, just based on the number of missing values or data leakage
* Transform features into the proper format (numerical to categorical, scaling numerical, filling in missing values, etc)
* Create new features by combining other features

In [88]:
def transform_features(df):
    '''
    1. Takes in a dataframe and returns a transformed version
    2. First drops any column with more than 5% of missing values
    3. For text columns, drops those with any missing values
    4. For numeric columns, fills in missing values with the the mode of that column
    5. Transforms 'Yr Sold' and 'Year Built' into 'Years Before Sale' and 'Years Since Remod'
    6. Drops non-useful columnns or those that leak data about the final sale
    '''
    
    # 2 - drops columns with more than 5% of missing values
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df)/20)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    # 3 - drops text columns with any missing values
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    # 4 - finds columns with missing values and fills the missing values with the mode
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    # 5 - Creates new columns that creates information from other columns and drops dirty negative values
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)

    # 6 - Drops non-useful columns or columns that leak data about the final sale
    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis=1)
    return df

def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

def train_and_test(df):  
    train = df[:1460]
    test = df[1460:]
    
    # You can use `pd.DataFrame.select_dtypes()` to specify column types
    # and return only those columns as a data frame.
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    # You can use `pd.Series.drop()` to drop a value.
    features = numeric_train.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

55275.36731241307

## Feature Selection

Now that we've cleaned and transformed a lot of the features in the data set, we will move on to selecting the features to use for the model for numerical features.

First, we will analyze correlation and variance to see default values for our pipeline.

Below we look at correlation.

In [89]:
# Create numerical df to do a correlation matrix
numerical_df = transform_df.select_dtypes(include=['int','float'])
absolute_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()

absolute_corr_coeffs

BsmtFin SF 2         0.006127
Misc Val             0.019273
Yr Sold              0.030358
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice 

As we can see above, we can see strong correlations in Gr Liv Area and Overall Qual and moderate correlations for quite a few. Looking at the statistics for the above:

In [90]:
absolute_corr_coeffs.describe()

count    34.000000
mean      0.338067
std       0.268350
min       0.006127
25%       0.104225
50%       0.280564
75%       0.543335
max       1.000000
Name: SalePrice, dtype: float64

Looking at above, we can keep the top 25% of values (those with an absolute correlation coefficient of 0.543335) to train on because they are moderate to strongly correlated. We can get by using 0.5 as the cutoff in this case.

In [91]:
transform_df = transform_df.drop(absolute_corr_coeffs[absolute_corr_coeffs < 0.5].index, axis=1)

Now, let's look at the columns meant to be categorical to see which should be kept and converted.

Here we will be looking to see which of these categorical ones provide variance to our model and would be useful to analyze.

In [92]:
# Starting with our original set of features
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

# Selecting columns that still exist in our transform
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)
        
# Selecting an arbitrary cutoff of unique values        
unique_counts = transform_df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
freq_counts = transform_df[transform_cat_cols].apply(lambda col: col.value_counts()[0])

# Putting both counts together
counts = pd.concat([unique_counts.rename('unique'),freq_counts.rename('mode_count')], axis=1)

counts

,unique,mode_count
Bldg Type,5,2422
Central Air,2,2731
Condition 1,9,2520
Condition 2,8,2898
Exterior 1st,16,1025
Exterior 2nd,17,1014
Foundation,6,1307
Heating,6,2882
House Style,8,1480
Land Contour,4,2632


Looking at the above, let's look at the statistics for above. In theory we should see which columns have few unique values where 1 value does not dominate more than 95% of the values.

In [93]:
counts.describe()

,unique,mode_count
count,16.000000,16.000000
mean,8.562500,2117.562500
std,6.632935,802.338496
min,2.000000,443.000000
25%,5.000000,1436.750000
50%,6.500000,2371.000000
75%,8.250000,2768.750000
max,28.000000,2915.000000


From above, let's generalize the average of the unique as a good count (so any column with less than 10 should be a good default).

For the mode_count, we will be using the 95% thesis, anything above 2783.5 values will be removed. Both of these fit with removing anything above our 75th percentile.

Now that we have an initial analysis to set our defaults, let us update select_features() and from there we can train and test the new data frame.

In [94]:
def select_features(df, coeff_threshold=0.5, uniq_threshold=10):
    # Create numerical dataframe and drop below coefficient threshold
    numerical_df = df.select_dtypes(include=['int','float'])
    absolute_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(absolute_corr_coeffs[absolute_corr_coeffs < coeff_threshold].index, axis=1)
        
    # Starting with our original set of features
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

    # Selecting columns that still exist in our transform
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)
        
    # Selecting an arbitrary cutoff of unique values        
    unique_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    freq_counts = df[transform_cat_cols].apply(lambda col: col.value_counts()[0])

    # Putting both counts together
    counts = pd.concat([unique_counts.rename('unique'),freq_counts.rename('mode_count')], axis=1)
    
    # Total length of dataframe
    length = df.shape[0]

    # Drop columns with too many unique values and too little variance
    df = df.drop(counts[((counts['unique'] > uniq_threshold) & (counts['mode_count'] > length*.95))].index, axis=1)

    # Select remaining text columns to convert to categorical
    text_cols = transform_df.select_dtypes(include=['object'])

    for col in text_cols:
        df[col] = df[col].astype('category')
                                     
    # Create dummy columns
    df = pd.concat(
        [df, pd.get_dummies(transform_df.select_dtypes(include=['category']))
        ], axis=1).drop(text_cols, axis=1)
    
    return df

## Train and Test

Now, we will finalize the training and testing pipeline. 

In [95]:
def train_and_test(df, k=0):  
    
    # You can use `pd.DataFrame.select_dtypes()` to specify column types
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    
    if k == 0:
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    if k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

ValueError: No objects to concatenate